In [8]:
import joblib
import ast
import pandas as pd

# -----------------------------
# Load Model & Encoders
# -----------------------------
model = joblib.load("disease_model.pkl")
symptom_encoder = joblib.load("symptom_encoder.pkl")
disease_encoder = joblib.load("disease_encoder.pkl")

# -----------------------------
# Load Treatment Info
# -----------------------------
def parse_treatment_file(path):
    treatment_map = {}
    current_disease = None

    with open(path, "r") as file:
        for line in file:
            line = line.strip()
            if not line:
                continue
            if line.endswith(":") and "Medicines" not in line:
                current_disease = line[:-1]
                treatment_map[current_disease] = {}
            elif current_disease:
                if line.startswith("Medicines:"):
                    treatment_map[current_disease]["medicines"] = line.replace("Medicines:", "").strip()
                elif line.startswith("Steps:"):
                    treatment_map[current_disease]["steps"] = line.replace("Steps:", "").strip()
                elif line.startswith("Duration:"):
                    treatment_map[current_disease]["duration"] = line.replace("Duration:", "").strip()
    return treatment_map

treatment_map = parse_treatment_file("treatments.txt")

def normalize_disease_name(name):
    name = name.lower().strip()
    if "(" in name and ")" in name:
        name = name.split(")")[-1].strip()
    return name

normalized_treatment_map = {
    normalize_disease_name(d): v for d, v in treatment_map.items()
}

# Load dataset
df = pd.read_csv("data.csv")

# -----------------------------
# Start Interaction
# -----------------------------
all_symptoms = []

while True:
    print("\n➕ Add new symptoms to refine the prediction (comma-separated):")
    new_input = input("> ")
    new_symptoms = [s.strip().lower() for s in new_input.split(",")]

    all_symptoms.extend(sym for sym in new_symptoms if sym not in all_symptoms)
    print(f"\n🧾 Current symptoms so far: {', '.join(all_symptoms)}")

    # -----------------------------
    # Strict Matching with Grouped Diseases (No Duplicates)
    # -----------------------------
    disease_to_extras = {}

    for _, row in df.iterrows():
        try:
            row_symptoms = ast.literal_eval(row["Symptoms"])
            normalized = set(s.strip().replace("_", " ").lower() for s in row_symptoms)
            if set(all_symptoms).issubset(normalized):
                disease = row["Disease"]
                extra = normalized - set(all_symptoms)
                if disease not in disease_to_extras:
                    disease_to_extras[disease] = {"overlap": set(all_symptoms), "extra": extra}
                else:
                    disease_to_extras[disease]["extra"].update(extra)
        except:
            continue

    if not disease_to_extras:
        print("❌ No disease contains all given symptoms.")
    else:
        print("\n🧠 Diseases that match ALL current symptoms:")
        for disease, data in disease_to_extras.items():
            print(f"\n🔸 {disease}")
            print(f"   ✓ Your Symptoms: {', '.join(data['overlap'])}")
            if data["extra"]:
                print(f"   ➕ Other symptoms typically seen: {', '.join(list(data['extra'])[:3])}")
            else:
                print("   ✅ All symptoms matched exactly.")

        # -----------------------------
        # Model Prediction + Treatment
        # -----------------------------
        valid_symptoms = set(symptom_encoder.classes_)
        filtered = [s for s in all_symptoms if s in valid_symptoms]

        if filtered:
            X_input = symptom_encoder.transform([filtered])
            y_pred = model.predict(X_input)
            predicted = disease_encoder.inverse_transform(y_pred)[0]
            norm_pred = normalize_disease_name(predicted)

            print(f"\n🎯 Most Likely Prediction: {predicted}")
            if norm_pred in normalized_treatment_map:
                treatment = normalized_treatment_map[norm_pred]
                print("\n💊 Medicines:", treatment.get("medicines", "Not specified"))
                print("🧾 Steps:", treatment.get("steps", "Not specified"))
                print("⏱️ Duration:", treatment.get("duration", "Not specified"))
            else:
                print("⚠️ No treatment info found.")
        else:
            print("⚠️ None of your symptoms matched the model vocabulary.")

    again = input("\n🔁 Add more symptoms to refine? (y/n): ").strip().lower()
    if again != "y":
        print("👋 Session complete. Thank you!")
        break


/opt/anaconda3/envs/mednlp/lib/python3.10/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.5.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/anaconda3/envs/mednlp/lib/python3.10/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.5.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/anaconda3/envs/mednlp/lib/python3.10/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator MultiLabelBinarizer from version


➕ Add new symptoms to refine the prediction (comma-separated):


>  headache, nausea, vomiting



🧾 Current symptoms so far: headache, nausea, vomiting

🧠 Diseases that match ALL current symptoms:

🔸 Malaria
   ✓ Your Symptoms: nausea, vomiting, headache
   ➕ Other symptoms typically seen: diarrhoea, sweating, muscle pain

🔸 Dengue
   ✓ Your Symptoms: nausea, vomiting, headache
   ➕ Other symptoms typically seen: fatigue, back pain, pain behind the eyes

🔸 Typhoid
   ✓ Your Symptoms: nausea, vomiting, headache
   ➕ Other symptoms typically seen: fatigue, diarrhoea, abdominal pain

🔸 Hypoglycemia
   ✓ Your Symptoms: nausea, vomiting, headache
   ➕ Other symptoms typically seen: fatigue, sweating, slurred speech

🔸 (vertigo) Paroymsal  Positional Vertigo
   ✓ Your Symptoms: nausea, vomiting, headache
   ➕ Other symptoms typically seen: loss of balance, spinning movements, unsteadiness

🎯 Most Likely Prediction: (vertigo) Paroymsal  Positional Vertigo

💊 Medicines: Meclizine, betahistine
🧾 Steps: Perform Epley maneuver. Avoid sudden head movements. Vestibular rehab if persistent.
⏱️ 


🔁 Add more symptoms to refine? (y/n):  y



➕ Add new symptoms to refine the prediction (comma-separated):


>  diarrhoea, sweating



🧾 Current symptoms so far: headache, nausea, vomiting, diarrhoea, sweating

🧠 Diseases that match ALL current symptoms:

🔸 Malaria
   ✓ Your Symptoms: diarrhoea, headache, sweating, nausea, vomiting
   ➕ Other symptoms typically seen: high fever, chills, muscle pain

🎯 Most Likely Prediction: Malaria

💊 Medicines: artemisinin-based combination therapy (ACT), intravenous quinine, paracetamol
🧾 Steps: Take ACT twice daily for 3 days. In severe cases, use intravenous quinine. Stay hydrated and manage fever.
⏱️ Duration: 7–10 days



🔁 Add more symptoms to refine? (y/n):  n


👋 Session complete. Thank you!
